In [1]:
import findspark
import time

from pyspark import SparkFiles
from pyspark.sql import SparkSession

findspark.init()
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [2]:
# Read the AWS S3 bucket into a DataFrame
url = 'https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv'
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get('home_sales_revised.csv'), sep=',', header=True)

In [3]:
# Create a temporary view of the DataFrame
home_sales_df.createOrReplaceTempView('home_sales')

In [4]:
# Calculate the average price for a four bedroom house sold per year, rounded to two decimal places
spark.sql('''
SELECT YEAR(date) AS sale_year, ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY Sale_Year
ORDER BY Sale_Year
''').show()

+---------+-------------+
|sale_year|average_price|
+---------+-------------+
|     2019|     300263.7|
|     2020|    298353.78|
|     2021|    301819.44|
|     2022|    296363.88|
+---------+-------------+



In [5]:
# Calculate the average price of a home for each year the home was built, that have 3 bedrooms and 3 bathrooms, rounded to two decimal places
spark.sql('''
SELECT date_built, ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built
''').show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [6]:
# Calculate the average price of a home for each year the home was built, that have 3 bedrooms, 3 bathrooms, 
# with two floors, and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql('''
SELECT date_built, ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built
''').show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [7]:
# Calculate the average price of a home per "view" rating, rounded to two decimal places, having 
# an average home price greater than or equal to $350,000. Order by descending view rating. 
# Determine the run time for this query.

start_time = time.time()

spark.sql('''
SELECT view, ROUND(AVG(price), 2) AS average_price
FROM home_sales
GROUP BY view
HAVING average_price >= 350000
ORDER BY INT(view) DESC
''').show()

print("--- Query run time: %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- Query run time: 0.6741971969604492 seconds ---


In [8]:
# Cache the the temporary view
spark.sql('CACHE TABLE home_sales')
spark.catalog.isCached('home_sales')

True

In [9]:
# Repeat the most recent query using the cached data. Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql('''
SELECT view, ROUND(AVG(price), 2) AS average_price
FROM home_sales
GROUP BY view
HAVING average_price >= 350000
ORDER BY INT(view) DESC
''').show()

print("--- Cached query run time: %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- Cached query run time: 0.3610365390777588 seconds ---


In [10]:
# Partition by the "date_built" field on the formatted parquet home sales data 
home_sales_df.write.partitionBy('date_built').mode('overwrite').parquet('home_sales_partitioned')

Py4JJavaError: An error occurred while calling o49.parquet.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.getAllCommittedTaskPaths(Fil

In [ ]:
# Read the formatted parquet data.


In [ ]:
# 12. Create a temporary table for the parquet data.


In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 14. Uncache the home_sales temporary table.


In [ ]:
# 15. Check if the home_sales is no longer cached

